In [ ]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field

# Option A: Using the @tool decorator with a docstring for description
@tool
def calculate_percentage_marks(marks_obtained: int, marks_total: int) -> float:
    """Calculate Percentage using Marks Obtained and Total Marks"""
    percentage = (marks_obtained * 100.0) / marks_total
    return round(percentage, 2)

# Option B: Using Pydantic for more structured argument schema
class AddInput(BaseModel):
    a: int = Field(description="First integer to add")
    b: int = Field(description="Second integer to add")

@tool(args_schema=AddInput)
def add_numbers(a: int, b: int) -> int:
    """Adds two integers together and returns the result."""
    return a + b

tools = [calculate_percentage_marks, add_numbers]


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI # Or other compatible model, e.g., ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

# Ensure you have the necessary API key set in your environment variables
llm = ChatOpenAI(model="gpt-4o", temperature=0) 

# Bind the tools to the LLM
llm_with_tools = llm.bind_tools(tools)


In [ ]:
from langchain_core.messages import ToolMessage
# 1. Invoke the LLM with a query that requires a tool
messages = [HumanMessage(content="What is the percentage of 670 marks out of a total of 850?")]
response = llm_with_tools.invoke(messages)
# print(response.tool_calls) 
# Example output will show a tool call request, e.g., [{'id': '...', 'name': 'calculate_percentage_marks', 'args': {'marks_obtained': 670, 'marks_total': 850}}]
tool_messages = []
# 2. Manually execute the tool call(s)
if response.tool_calls:
    for tool_call in response.tool_calls:
        # Find the correct tool function and execute it with arguments
        # A simple way to map name to function is a dictionary
        # print(tool_call)
        tool_map = {tool.name: tool for tool in tools}
        selected_tool = tool_map[tool_call["name"]]
        tool_output = selected_tool.invoke(tool_call["args"])
        
        # 3. Create a ToolMessage with the output and append to messages
        tool_messages.append(ToolMessage(content=str(tool_output), tool_call_id=tool_call["id"]))
    
# 4. Invoke the model again with the original prompt AND the tool output
final_response = llm_with_tools.invoke([HumanMessage(content="What is the percentage of 670 marks out of a total of 850?"),response]+tool_messages)
print(final_response.content)


The percentage of 670 marks out of a total of 850 is 78.82%.
